In [29]:
# Import the data
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [30]:
df = pd.read_csv('Resources/Customer-Churn-Records.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1,1,2,DIAMOND,464
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,1,3,DIAMOND,456
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,1,3,DIAMOND,377
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0,5,GOLD,350
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,0,5,GOLD,425


In [31]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited', 'Complain',
       'Satisfaction Score', 'Card Type', 'Point Earned'],
      dtype='object')

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   RowNumber           10000 non-null  int64  
 1   CustomerId          10000 non-null  int64  
 2   Surname             10000 non-null  object 
 3   CreditScore         10000 non-null  int64  
 4   Geography           10000 non-null  object 
 5   Gender              10000 non-null  object 
 6   Age                 10000 non-null  int64  
 7   Tenure              10000 non-null  int64  
 8   Balance             10000 non-null  float64
 9   NumOfProducts       10000 non-null  int64  
 10  HasCrCard           10000 non-null  int64  
 11  IsActiveMember      10000 non-null  int64  
 12  EstimatedSalary     10000 non-null  float64
 13  Exited              10000 non-null  int64  
 14  Complain            10000 non-null  int64  
 15  Satisfaction Score  10000 non-null  int64  
 16  Card 

In [33]:
df['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [34]:
df['Card Type'].unique()

array(['DIAMOND', 'GOLD', 'SILVER', 'PLATINUM'], dtype=object)

**Target Value**  
- Exited  

**Values to be Removed**  
- Remove index rows: `RowNumber` and `CustomerId`  
- Remove `Surname`  

**Numerical X Values**  
- Credit Score  
- Age  
- Tenure  
- Balance  
- NumOfProducts  
- HasCrCard  
- IsActiveMember  
- EstimatedSalary  
- Complain  
- Satisfaction Score  
- Points Earned  

**Encoded X Variables**  
- Geography  
- Gender  

**Ordinal X**  
- Card Type

In [35]:
encoder = OneHotEncoder(sparse_output=False)
ordinal_encoder = OrdinalEncoder(categories=['SILVER', 'GOLD', 'PLATINUM', 'DIAMOND'])

In [36]:
categorical_columns = ['Geography', 'Gender']
ordinal_columns = ['Card Type']

In [37]:
encoded_columns = encoder.fit_transform(df[categorical_columns])
#Create a DataFrame with the encoded columns

one_hot_df = pd.DataFrame(encoded_columns, 
                          columns=encoder.get_feature_names_out(categorical_columns))

# Concatenate the one-hot encoded columns with the original DataFrame
df_encoded = pd.concat([df.drop(categorical_columns, axis=1), one_hot_df], axis=1)
df_encoded.head()


,RowNumber,CustomerId,Surname,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,...,Exited,Complain,Satisfaction Score,Card Type,Point Earned,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,1,15634602,Hargrave,619,42,2,0.00,1,1,1,...,1,1,2,DIAMOND,464,1.0,0.0,0.0,1.0,0.0
1,2,15647311,Hill,608,41,1,83807.86,1,0,1,...,0,1,3,DIAMOND,456,0.0,0.0,1.0,1.0,0.0
2,3,15619304,Onio,502,42,8,159660.80,3,1,0,...,1,1,3,DIAMOND,377,1.0,0.0,0.0,1.0,0.0
3,4,15701354,Boni,699,39,1,0.00,2,0,0,...,0,0,5,GOLD,350,1.0,0.0,0.0,1.0,0.0
4,5,15737888,Mitchell,850,43,2,125510.82,1,1,1,...,0,0,5,GOLD,425,0.0,0.0,1.0,1.0,0.0


In [38]:
ordinal_encoder = OrdinalEncoder(categories=[['SILVER', 'GOLD', 'PLATINUM', 'DIAMOND']])
ordinal_column = ordinal_encoder.fit_transform(df[['Card Type']])
ordinal_df = pd.DataFrame(ordinal_column, 
                          columns=ordinal_encoder.get_feature_names_out(['Card Type']))

# Concatenate the one-hot encoded columns with the original DataFrame
df_encoded = pd.concat([df_encoded.drop(['Card Type'], axis=1), ordinal_df], axis=1)
df_encoded.head()

,RowNumber,CustomerId,Surname,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,...,Exited,Complain,Satisfaction Score,Point Earned,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male,Card Type
0,1,15634602,Hargrave,619,42,2,0.00,1,1,1,...,1,1,2,464,1.0,0.0,0.0,1.0,0.0,3.0
1,2,15647311,Hill,608,41,1,83807.86,1,0,1,...,0,1,3,456,0.0,0.0,1.0,1.0,0.0,3.0
2,3,15619304,Onio,502,42,8,159660.80,3,1,0,...,1,1,3,377,1.0,0.0,0.0,1.0,0.0,3.0
3,4,15701354,Boni,699,39,1,0.00,2,0,0,...,0,0,5,350,1.0,0.0,0.0,1.0,0.0,1.0
4,5,15737888,Mitchell,850,43,2,125510.82,1,1,1,...,0,0,5,425,0.0,0.0,1.0,1.0,0.0,1.0


In [39]:
correlation = df_encoded.drop(columns='Surname').corr()['Exited']
display(correlation.sort_values(ascending=False))

Exited                1.000000
Complain              0.995693
Age                   0.285296
Geography_Germany     0.173313
Balance               0.118577
Gender_Female         0.106267
Card Type             0.016949
EstimatedSalary       0.012490
Point Earned         -0.004628
Satisfaction Score   -0.005849
CustomerId           -0.006203
HasCrCard            -0.006976
Tenure               -0.013656
RowNumber            -0.016140
CreditScore          -0.026771
NumOfProducts        -0.047611
Geography_Spain      -0.052800
Geography_France     -0.104688
Gender_Male          -0.106267
IsActiveMember       -0.156356
Name: Exited, dtype: float64

In [40]:
X = df_encoded.drop(columns=[ 'RowNumber', 'CustomerId', 'Surname', 'Exited', 'Complain'])

In [41]:
y = df['Exited']

In [42]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [43]:
scaler_train = StandardScaler()
scaler_test = StandardScaler()
scaler_train.fit(X_train)
scaler_test.fit(X_test)
X_train_scaled = scaler_train.transform(X_train)
X_test_scaled = scaler_test.transform(X_test)

In [45]:
# Initialize the LogisticRegression model
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression()

In [46]:
# Fit the model to the training data
lr_model.fit(X_train_scaled, y_train)

LogisticRegression()

In [47]:
# Validate the model by checking the model accuracy with model.score
print('Train Accuracy: %.3f' % lr_model.score(X_train_scaled, y_train))
print('Test Accuracy: %.3f' % lr_model.score(X_test_scaled, y_test))

Train Accuracy: 0.811
Test Accuracy: 0.812


In [48]:
# Make and save testing predictions with the saved SVM model using the testing data
testing_predictions = lr_model.predict(X_test_scaled)

# Review the predictions
testing_predictions

array([0, 0, 0, ..., 0, 0, 0])

In [49]:
# Display the accuracy score for the testing dataset
accuracy_score(y_test, testing_predictions)

0.812

In [50]:
# Compare to balanced accuracy score
score = balanced_accuracy_score(y_test, testing_predictions)
print('Balanced Accuracy Score:', score)

Balanced Accuracy Score: 0.5841016798326659


In [57]:
param_grid = {
    'C': [0.1, 1, 10, 100],
    'solver': ['liblinear', 'saga'],
}

In [58]:
# Create a GridSearchCV object
grid_search = GridSearchCV(lr_model, param_grid, cv=5, scoring='balanced_accuracy')

In [59]:
# Fit the model
grid_search.fit(X_train_scaled, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.1, 1, 10, 100],
                         'solver': ['liblinear', 'saga']},
             scoring='balanced_accuracy')

In [60]:
# View the best parameters
print("Best parameters found: ", grid_search.best_params_)


Best parameters found:  {'C': 1, 'solver': 'saga'}


In [61]:
# Evalueate the model
from sklearn.metrics import classification_report
gscv_predictions = grid_search.predict(X_test_scaled)
print(classification_report(y_test, gscv_predictions))

              precision    recall  f1-score   support

           0       0.83      0.96      0.89      1607
           1       0.56      0.21      0.30       393

    accuracy                           0.81      2000
   macro avg       0.69      0.58      0.60      2000
weighted avg       0.78      0.81      0.78      2000



In [64]:
# Define and fit model with optimal parameters
lr_refined = LogisticRegression(C= 1, solver='saga')
lr_refined.fit(X_train_scaled, y_train)

LogisticRegression(C=1, solver='saga')

In [66]:
print('Train Accuracy: %.3f' % lr_refined.score(X_train_scaled, y_train))
print('Test Accuracy: %.3f' % lr_refined.score(X_test_scaled, y_test))

Train Accuracy: 0.811
Test Accuracy: 0.812


In [67]:
# Make and save testing predictions with the saved LR Refined model using the testing data
testing_predictions = lr_refined.predict(X_test_scaled)

# Review the predictions
testing_predictions

array([0, 0, 0, ..., 0, 0, 0])

In [68]:
# Compare to balanced accuracy score
score = balanced_accuracy_score(y_test, testing_predictions)
print('Balanced Accuracy Score:', score)

Balanced Accuracy Score: 0.5841016798326659
